# 🛠 Exercises

## 1. Spend 20 minutes reading and interacting with the [CNN explainer website](https://poloclub.github.io/cnn-explainer/).
- What are the key terms? e.g. explain convolution in your own words, pooling in your own words

Convolution is a method to extract features from the image to learn patterns to distinguish images. It does element-wise product of the kernel size and filter depth to get different numbers, where the kernel is a learned patterns based on the image. Earlier layers of the convolution extract tiny parts of the image (e.g. edges/lines) and later layers of the convolution extract bigger part of the image (e.g. human's face).

Pooling compresses the image by taking the numbers that meet the certain criteria (i.e. take the necessary information and ignore the rest) (e.g. taking the maximum number by filtering the image with kernels) and also reduces the number of parameters to be trained by reducing the size of the convolution.

## 2. Play around with the "understanding hyperparameters" section in the [CNN explainer website](https://poloclub.github.io/cnn-explainer/) for 10-minutes.
- What is the kernel size?
    - A hyperparameter to set the size of the weights the network will learn and update.
    - Small kernel size leads to better performance than large kernel size since we can stack layers and learn complex features
- What is the stride?
    - A hyperparameter that sets how many pixels a kernel should move over while performing convolution
    - Similar to setting kernel size, a small stride leads to extracting more features than a large stride
- How could you adjust each of these in TensorFlow code?
    - Code for convolution contains options to set the kernel size and the stride
    - `tf.keras.layers.Conv2D(..., kernel_size=, stride=)`

## 3. Take 10 photos of two different things and build your own CNN image classifier using the techniques we've built here.

In [1]:
# Load libraries
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Load binary classification dataset
import tensorflow_datasets as tfds

(train_ds, test_ds), ds_info = tfds.load(name='horses_or_humans',
                                         data_dir='tmp',
                                         as_supervised=True,
                                         with_info=True,
                                         split=['train', 'test'])

train_ds, test_ds

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1027 [00:00<?, ? examples/s]

Shuffling tmp/horses_or_humans/3.0.0.incompleteYMCMCY/horses_or_humans-train.tfrecord*...:   0%|          | 0/…

Generating test examples...:   0%|          | 0/256 [00:00<?, ? examples/s]

Shuffling tmp/horses_or_humans/3.0.0.incompleteYMCMCY/horses_or_humans-test.tfrecord*...:   0%|          | 0/2…

Dataset horses_or_humans downloaded and prepared to tmp/horses_or_humans/3.0.0. Subsequent calls will reuse this data.


(<_PrefetchDataset element_spec=(TensorSpec(shape=(300, 300, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
 <_PrefetchDataset element_spec=(TensorSpec(shape=(300, 300, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>)

In [3]:
# Check the dataset info
ds_info

tfds.core.DatasetInfo(
    name='horses_or_humans',
    full_name='horses_or_humans/3.0.0',
    description="""
    A large set of images of horses and humans.
    """,
    homepage='http://laurencemoroney.com/horses-or-humans-dataset',
    data_dir=PosixGPath('/tmp/tmpmuld6h15tfds'),
    file_format=tfrecord,
    download_size=153.59 MiB,
    dataset_size=153.53 MiB,
    features=FeaturesDict({
        'image': Image(shape=(300, 300, 3), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=2),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=256, num_shards=1>,
        'train': <SplitInfo num_examples=1027, num_shards=2>,
    },
    citation="""@ONLINE {horses_or_humans,
    author = "Laurence Moroney",
    title = "Horses or Humans Dataset",
    month = "feb",
    year = "2019",
    url = "http://laurencemoroney.com/horses-or-humans-dataset"
    }""",
)

In [4]:
# Preprocessing function
def format_example(image, label):
    # Make image color values to be float
    image = tf.cast(image, tf.float32)
    # Make image color values to be normalized
    image /= 255
    # Make sure that image has a right size
    image = tf.image.resize(image, [300, 300])

    return image, label

In [5]:
# Preprocess data
dataset_train = train_ds.map(format_example)
dataset_test = test_ds.map(format_example)

dataset_train, dataset_test

(<_MapDataset element_spec=(TensorSpec(shape=(300, 300, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
 <_MapDataset element_spec=(TensorSpec(shape=(300, 300, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>)

In [6]:
# Put data into batches
dataset_train = dataset_train.batch(batch_size=32)
dataset_test = dataset_test.batch(batch_size=32)

dataset_train, dataset_test

(<_BatchDataset element_spec=(TensorSpec(shape=(None, 300, 300, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>,
 <_BatchDataset element_spec=(TensorSpec(shape=(None, 300, 300, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>)

In [7]:
# Prefetch will enable the input pipeline to asynchronously fetch batches while your model is training.
dataset_train = dataset_train.prefetch(buffer_size = tf.data.experimental.AUTOTUNE)

dataset_train

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 300, 300, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [8]:
# Set up a model
model_1 = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation = 'relu', input_shape = (300, 300, 3)),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(32, 3, activation = 'relu'),
    tf.keras.layers.MaxPool2D(),
    # tf.keras.layers.Conv2D(32, 3, activation = 'relu'),
    # tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

# Compile the model
model_1.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

# Train the model
history_1 = model_1.fit(dataset_train,
                        batch_size=32,
                        validation_data=dataset_test,
                        epochs=20)

Epoch 1/20
33/33 [==============================] - 17s 131ms/step - loss: 0.5748 - accuracy: 0.7702 - val_loss: 0.9827 - val_accuracy: 0.7109
Epoch 2/20
33/33 [==============================] - 2s 51ms/step - loss: 0.0737 - accuracy: 0.9805 - val_loss: 2.0205 - val_accuracy: 0.7070
Epoch 3/20
33/33 [==============================] - 2s 54ms/step - loss: 0.0202 - accuracy: 0.9961 - val_loss: 2.5542 - val_accuracy: 0.7109
Epoch 4/20
33/33 [==============================] - 2s 68ms/step - loss: 0.0071 - accuracy: 1.0000 - val_loss: 2.9927 - val_accuracy: 0.6836
Epoch 5/20
33/33 [==============================] - 2s 52ms/step - loss: 0.0059 - accuracy: 0.9990 - val_loss: 2.3374 - val_accuracy: 0.7461
Epoch 6/20
33/33 [==============================] - 2s 51ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 2.7428 - val_accuracy: 0.7305
Epoch 7/20
33/33 [==============================] - 2s 57ms/step - loss: 7.5345e-04 - accuracy: 1.0000 - val_loss: 2.6987 - val_accuracy: 0.7383
Epoch 8

## 4. Find an ideal learning rate for a simple convolutional neural network model on the 10-class dataset.

In [9]:
# Load fashion mnist dataset
from tensorflow.keras.datasets import fashion_mnist

(train_X, train_y), (test_X, test_y) = fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [10]:
# Normalize data
train_X = train_X / 255
test_X = test_X / 255

In [11]:
# Configure learning rate scheduler
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-4 * 10 ** (epoch / 20))

In [12]:
# Set up a model
model_2 = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, padding = 'same', activation = 'relu', input_shape = (28, 28, 1)),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(32, 3, padding = 'same', activation = 'relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(32, 3, padding = 'same', activation = 'relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation = 'softmax')
])

# Compile the model
model_2.compile(loss='sparse_categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

# Train the model
history_2 = model_2.fit(train_X,
                        train_y,
                        batch_size=32,
                        epochs=20,
                        validation_data=(test_X, test_y),
                        callbacks=[lr_scheduler])

Epoch 1/20
1875/1875 [==============================] - 10s 4ms/step - loss: 0.7414 - accuracy: 0.7373 - val_loss: 0.5226 - val_accuracy: 0.8045 - lr: 1.0000e-04
Epoch 2/20
1875/1875 [==============================] - 8s 4ms/step - loss: 0.4595 - accuracy: 0.8367 - val_loss: 0.4409 - val_accuracy: 0.8439 - lr: 1.1220e-04
Epoch 3/20
1875/1875 [==============================] - 7s 4ms/step - loss: 0.4077 - accuracy: 0.8561 - val_loss: 0.4059 - val_accuracy: 0.8540 - lr: 1.2589e-04
Epoch 4/20
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3778 - accuracy: 0.8671 - val_loss: 0.3889 - val_accuracy: 0.8594 - lr: 1.4125e-04
Epoch 5/20
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3532 - accuracy: 0.8756 - val_loss: 0.3701 - val_accuracy: 0.8659 - lr: 1.5849e-04
Epoch 6/20
1875/1875 [==============================] - 9s 5ms/step - loss: 0.3336 - accuracy: 0.8813 - val_loss: 0.3571 - val_accuracy: 0.8704 - lr: 1.7783e-04
Epoch 7/20
1875/1875 [===========

# 📖 Extra-curriculum

1. **Watch**: [MIT's Introduction to Deep Computer Vision](https://www.youtube.com/watch?v=iaSUYvmCekI&list=PLtBw6njQRU-rwp5__7C0oIVt26ZgjG9NI&index=3) lecture. This will give you a great intuition behind convolutional neural networks.

2. **Watch**: Deep dive on [mini-batch gradient descent](https://youtu.be/-_4Zi8fCZO4) by deeplearning.ai. If you're still curious about why we use **batches** to train models, this technical overview covers many of the reasons why.

3. **Read**: [CS231n Convolutional Neural Networks for Visual Recognition](https://cs231n.github.io/convolutional-networks/) class notes. This will give a very deep understanding of what's going on behind the scenes of the convolutional neural network architectures we're writing.

4. **Read**: ["A guide to convolution arithmetic for deep learning"](https://arxiv.org/pdf/1603.07285.pdf). This paper goes through all of the mathematics running behind the scenes of our convolutional layers.

5. **Code practice**: [TensorFlow Data Augmentation Tutorial](https://www.tensorflow.org/tutorials/images/data_augmentation). For a more in-depth introduction to data augmentation with TensorFlow, spend an hour or two reading through this tutorial.